In [5]:
import pandas as pd
from collections import deque
movies = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\movies.csv')
people = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\people.csv')
stars = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\stars.csv')
people.columns = ['person_id', 'name', 'birth']
movies.columns = ['movie_id', 'title', 'year']
stars = stars.merge(people, 'left', 'person_id').merge(movies, 'left', 'movie_id')
stars = stars[['name', 'title']]
stars.columns = ['actor', 'movie']

In [6]:


class Network:
    def __init__(self, actor, stars):
        self.actor = actor
        self.stars = stars
        self.actor_to_movies = stars.groupby('actor')['movie'].apply(set).to_dict()
        self.movie_to_actors = stars.groupby('movie')['actor'].apply(set).to_dict()
        initial_movies = self.actor_to_movies.get(actor, set())
        self.frontier = deque([Node(movie, self.movie_to_actors[movie], None, None) for movie in initial_movies])
        self.explored = set(initial_movies)
            
    def find_degrees(self, target_actor):
        while self.frontier:
            explored_node = self.frontier.popleft()
            if target_actor in explored_node.cast:
                return self.build_path(explored_node, target_actor)

            for movie in set.union(*(self.actor_to_movies.get(actor, set()) for actor in explored_node.cast)) - self.explored:
                self.explored.add(movie)
                new_node = Node(movie, self.movie_to_actors[movie], explored_node, actor)
                self.frontier.append(new_node)
        raise Exception('No connection found between actors')

    def build_path(self, node, target_actor):
        path = []
        while node:
            path.append((node.movie, node.parent_actor if node.parent_actor else target_actor))
            node = node.parent_movie
        return path[::-1]

class Node:
    def __init__(self, movie, cast, parent_movie, parent_actor):
        self.movie = movie
        self.cast = cast
        self.parent_movie = parent_movie
        self.parent_actor = parent_actor

# Usage
# network = Network('Kevin Bacon', stars)
# path = network.find_degrees('Tom Cruise')


In [7]:
Network('Brad Pitt', stars)
Network.find_degrees('Tom Cruise')

TypeError: Network.find_degrees() missing 1 required positional argument: 'target_actor'